# CAGIRE saturation study

This notebook aims at quantifying how many stars will saturate the central pixel of CAGIRE for a given Field of View and eposure time.   

Requirements:
- pyETC  
- ImSimpy  (if you want to simulate images)
- astroquery  
- ephem  
- Quaternion  



In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import os
from astroquery.vizier import Vizier
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.io import ascii
import astropy.coordinates as coord
from Quaternion import Quat
import ephem
from astropy.table import Table,Column,vstack
import imp
import warnings
warnings.filterwarnings('ignore')

_, path, _ = imp.find_module('ImSimpy')

In [ ]:
%load_ext autoreload
%autoreload 2

# Compute the number of e- created in the central pixel for a given magnitude

In [ ]:
# Load ETC package
from pyETC.pyETC import etc

# seeing at zenith at 550nm (arcsec)
seeing = 0.8

moonage=7

Nelec_saturation = 60000

# Set exposure time
Texps=np.linspace(1.,50)*1.4
#Texps=[10]
# Set bands
bands=['J','H']

telescope = ['colibri','colibri_teledyne']
for tel  in telescope:
    # load ETC with a config file and the GFT caracteristics
    colibri_ETC=etc(configFile='CAGIRE_saturation.hjson',
                name_telescope=tel)

    # Change the number of expositions
    colibri_ETC.information['Nexp']=1
    # Change the seeing at the zenith, in arcseconds
    colibri_ETC.information["seeing_zenith"]=seeing
    #Change the age of the moon
    colibri_ETC.information["moon_age"]=moonage
    # And also the channel in order to load the CAGIRE channel caracterisitcs (optics transmissions + camera carac.)
    colibri_ETC.information["channel"]= "CAGIRE"
    # Change the quantity to compute, now we want to compute the SNR
    colibri_ETC.information['etc_type']='snr'
    colibri_ETC.information['SNR']=5
    colibri_ETC.information['object_type']='magnitude'
    colibri_ETC.information['photometry_system']='Vega'

    for band in bands:
        
        result_table = []
        # Select the filter band
        colibri_ETC.information["filter_band"]= band
        
        for i,Texp in enumerate(Texps):
                
                # Modify the exposure time
                colibri_ETC.information["exptime"]= Texp
            
                colibri_ETC.sim()
                RN=colibri_ETC.information['cameras'][colibri_ETC.information['channel']]['RN']
                DC=colibri_ETC.information['cameras'][colibri_ETC.information['channel']]['DC']
                inst_bg=colibri_ETC.information["Instrument_bg"]
                DigN=colibri_ETC.information["dig_noise"]
                SB=colibri_ETC.information["Sky_CountRate"]
                ZP=colibri_ETC.information['zeropoint']
                f_pix=colibri_ETC.information['f_pix']

                # computes how many electrons can come from signal before saturation
                Nelec_source= Nelec_saturation -  Texp * ( DC + SB + inst_bg)
                
                # transform this number of e- in magnitude using zeropints and fraction of e- in central pixel
                Mag2Saturation=ZP - 2.5*np.log10(Nelec_source/f_pix/Texp)
                
                #tot_elec= (RN + DigN) + Texp * ( DC + SB + inst_bg)
                tot_elec= Texp * ( DC + SB + inst_bg)
                result_table.append([Texp,int(i+1),np.round(SB*Texp,2),np.round(DC*Texp,3),np.round(inst_bg*Texp,2),
                                     np.round(tot_elec,2),np.round(Nelec_source,2),np.round(Mag2Saturation,2)])

        results=Table(np.array(result_table),names=['Texp','Nexp','SB','DC','inst_bg','Total_elec','Nelec_source','Mag2Saturation'])
        print (results)
        results.write('table_%s_%s.tex' % (tel,band))

# Function to download a given FoV from the 2MASS catalog and compute how many stars are saturating the central pixel

In [ ]:
def check_NIR_saturation(RA,DEC,Texp,band,seeing=1,moon_age=7,Nelec_saturation=60000,frame='icrs',catalog='II/246',
                         name_telescope='colibri',configFile='CAGIRE_saturation.hjson',
                         output_dir='saturation',verbose=False):
    """
    Function to check how many stars will saturate for a given exposure in J or H band
    RA: right ascencion (in degrees)
    DEC: declination (in degrees)
    radius: field of view radius (in arcmin)
    frame: icrs by default
    catalog: II/246 by default (2MASS)
    """
    RA=np.atleast_1d(RA)
    DEC=np.atleast_1d(DEC)
    nb_saturated_stars=[]
    total_nb_stars=[]
    
    from astropy.table import vstack

    # Import ETC
    from pyETC.pyETC import etc
        
    colibri_ETC=etc(configFile=configFile,name_telescope=name_telescope)
    
    colibri_ETC.information['seeing_zenith']=seeing
    colibri_ETC.information['moon_age']=moon_age
    colibri_ETC.information['filter_band']=band
    colibri_ETC.information['exptime']=Texp
    colibri_ETC.information['photometry_system']='Vega'
    colibri_ETC.sim()
    
    RN=colibri_ETC.information['cameras'][colibri_ETC.information['channel']]['RN']
    DC=colibri_ETC.information['cameras'][colibri_ETC.information['channel']]['DC']
    DigN=colibri_ETC.information['dig_noise']
    BN = colibri_ETC.information['Sky_CountRate']
    inst_bg=colibri_ETC.information['Instrument_bg']
    ZP=colibri_ETC.information['zeropoint']
    f_pix=colibri_ETC.information['f_pix']
    
    if verbose == True: 
        print ('Pixel scale: %.2f arcsec'%colibri_ETC.information['pixelScale_X'])
        print ('Field of View: %.2f arcmin'%colibri_ETC.information['FoV_axis1'])
        print ('Saturation occuring with a total of %d e-' % Nelec_saturation)
    
    # We consider a box not a circle so this radius correspond to one axis length
    radius=colibri_ETC.information['FoV_axis1']
    
    signal_CR= Nelec_saturation - ((RN + DigN) + Texp * ( DC + BN + inst_bg))
    
    if signal_CR <= 0:
        if verbose == True: print ('Saturation. Sky background alone saturate pixels')
        SB_saturation=True
        Saturated_stars=None
    else:
        SB_saturation=False
        Mag2Saturation=ZP - 2.5*np.log10(signal_CR/f_pix/Texp)
        if verbose==True:
            print ('For exposure of %.2fs in %s band' % (Texp,band))
            print ('%de- from noise, %d e- from signal, corresponding to %.2f mag (AB)' % (Nelec_saturation-signal_CR,signal_CR,Mag2Saturation))
                # Vizier Query for 2MASS catalog using astroquery
       
        # 2MASS catalog is named II/246
        v = Vizier(columns=['RAJ2000', 'DEJ2000','Jmag', 'Hmag', "+_r"],catalog='II/246')
        v.ROW_LIMIT =-1
        first_result_table=True
        tot=len(RA)
        step=0
        for ra,dec in zip(RA,DEC):
            step+=1
            print ("%.2f percent completed" % (step/tot*100), end="\r")
            
            #result = v.query_region(SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame=frame),radius=radius*u.arcmin)
            result = v.query_region(SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame=frame),width=[radius*u.arcmin,radius*u.arcmin])
            
            mask = result[0]['%smag' % band] <= Mag2Saturation
            
            if mask.any():
                nb_saturated_stars.append([ra,dec,len(result[0][mask])])
            else:
                nb_saturated_stars.append([ra,dec,0])
                
            total_nb_stars.append(len(result[0]))
            
            if first_result_table == True: Saturated_stars=result[0][mask]
            else: Saturated_stars=vstack([Saturated_stars,result[0][mask]])
            first_result_table=False
    return (SB_saturation,Saturated_stars,nb_saturated_stars,total_nb_stars,Mag2Saturation)

# Load first year SVOM B1 law

In [ ]:
data=ascii.read('ephem_att_nominal_2022.txt')

Column description for file "ephem_att_nominal_2022.txt":  

Col1: Date measured in days from 01/01/1950  
Col2-5: Attitude Quaterni_on  
Col6: Flag : 0 or 1 if Earth is in MXT FOV (taking into account guard angle)  
Col7:Flag : 0 or 1 if Earth is in VT FOV (taking into account guard angle   
Col8: Flag giving the type of the current pointing  
    1 : B1  
    2.1 : GPP close to B1  
    2.2 : GPP far from B1  
    3.1 : ToO NOM for GRB revisit  
    3.2 : ToO NOM (other than GRB revisit)  
    3.3 : ToO EX (Exceptional ToO)  
    4.1    : first observation of type A GRB  
    4.2    : first observation of type B GRB   
    5 : pointing toward false GRB  
    When the satellite is slewing this flag is negative  
Col9: Flag giving the satellite or mission status:  
    0 : « nothing to declare)  
    -1 : satellite unavailable  
    -2 : instrument calibration  
    -3 : biais calibration  
    -4 : orbit control maneuver:   

In [ ]:
# Transform Quaternion coordinates in RA DEC

RADEC_list=[]
for date,w,x,y,z in data['col1','col2','col3','col4','col5']:
    quat = Quat((x,y,z,w))
    RADEC_list.append([date,quat.ra,quat.dec,quat.roll])
RADEC_list=np.array(RADEC_list)

# Delete duplicate FoV. Overlapping FoV are kept
RADEC_SVOM=RADEC_list[:,1:3]
RADEC_SVOM_unique=np.unique(RADEC_SVOM,axis=0)
#RADEC_SVOM_unique=np.array([list(t) for t in set(tuple(element) for element in RADEC_list[:,1:3])])


In [ ]:
# Convert in astropy angle
SVOM_B1_law_RA=coord.Angle(RADEC_list[:,1]*u.degree)
SVOM_B1_law_DEC=coord.Angle(RADEC_list[:,2]*u.degree)
#SVOM_B1_law_RA=coord.Angle(RADEC_unique[:,0]*u.degree)
#SVOM_B1_law_DEC=coord.Angle(RADEC_unique[:,1]*u.degree)

# Conversion in galactic coordinates
c_icrs = SkyCoord(ra=SVOM_B1_law_RA, dec=SVOM_B1_law_DEC, frame='icrs')
SVOM_B1_law_lat=coord.Angle(c_icrs.galactic.b.degree*u.degree)
SVOM_B1_law_lon=coord.Angle(c_icrs.galactic.l.degree*u.degree)

In [ ]:
#Galactic plane
lon_array = np.linspace(0,360,1000)
lat = 0.
eq_array = np.zeros((1000,2))
for i,lon in enumerate(lon_array):
    ga = ephem.Galactic(np.radians(lon), np.radians(lat))
    eq = ephem.Equatorial(ga)
    eq_array[i] = np.degrees(eq.get())
Galactic_plane_RA = coord.Angle(eq_array[:,0]*u.degree)
Galactic_plane_DEC = coord.Angle(eq_array[:,1]*u.degree)

In [ ]:
# Matplotlib only takes -180° to 180° and in radians for aitoff/mollweide projetion
SVOM_B1_law_RA2=SVOM_B1_law_RA.wrap_at(180*u.degree)
SVOM_B1_law_lon2=SVOM_B1_law_lon.wrap_at(180*u.degree)
Galactic_plane_RA2 = Galactic_plane_RA.wrap_at(180*u.degree)

In [ ]:
plt.figure()
plt.scatter(SVOM_B1_law_RA2,SVOM_B1_law_DEC,label='SVOM',s=1)
plt.scatter(Galactic_plane_RA2,Galactic_plane_DEC,color='red',label='galactic plane',s=1)
plt.xlabel('RA (J2000)')
plt.ylabel('DEC (J2000)')
plt.xlim(180,-180)
plt.ylim(-90,90)
plt.legend()
# make plots directory
os.mkdir('plots/')
plt.savefig('plots/SVOM_RADEC.png')

In [ ]:
fig = plt.figure(figsize=(8,6))
#ax = fig.add_subplot(111, projection="mollweide")
ax = fig.add_subplot(111, projection="aitoff")
ax.scatter(SVOM_B1_law_RA2.radian, SVOM_B1_law_DEC.radian,label='SVOM pointings',s=1)
ax.scatter(Galactic_plane_RA2.radian,Galactic_plane_DEC.radian,color='red',label='galactic plane',s=1)
ax.set_ylabel('DEC (J2000)')
ax.set_xlabel('RA (J2000)')
plt.grid(True)
plt.legend(loc='lower right', markerscale=6,fontsize=10)
ax.set_title('Equatorial coordinate system \n(Aitoff projection)\n')
plt.savefig('plots/SVOM_Equatorial_system.png')

In [ ]:
fig = plt.figure(figsize=(8,6))
#ax = fig.add_subplot(111, projection="mollweide")
ax = fig.add_subplot(111, projection="aitoff")
ax.scatter(SVOM_B1_law_lon2.radian, SVOM_B1_law_lat.radian,s=1)
ax.set_ylabel('Latitude')
ax.set_xlabel('Longitude')
#ax.set_xlim(180,-180)
plt.grid(True)
ax.set_title('Galactic coordinate system \n(Aitoff projection)\n')
plt.savefig('plots/SVOM_Galactic_system')

# Distribution of stars magnitude for all the SVOM pointings

In [ ]:
# Distribution of stars magnitude for all the pointings
SVOM_fov_Jmags=[]
SVOM_fov_Hmags=[]
RADEC_JH=[]
step=0
tot=len(RADEC_SVOM_unique[:,0])

v = Vizier(columns=['RAJ2000', 'DEJ2000','Jmag', 'Hmag', "+_r"],catalog='II/246')
# Get the whole table and not only the 50 first sources
v.ROW_LIMIT =-1
# Set the time limit before receiving a timeout error in case of a bad connexion 
Vizier.TIMEOUT=60

# CAGIRE FoV length (we consider a box not a circle)
radius=21.7

for ra,dec in zip(RADEC_SVOM_unique[:,0],RADEC_SVOM_unique[:,1]):
    
    step+=1
    print ("%.2f percent completed" % (step/tot*100), end="\r")
    #result = v.query_region(SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs'),radius=radius*u.arcmin)
    result = v.query_region(SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs'),width=[radius*u.arcmin,radius*u.arcmin])

    for i in range(len(result[0])):
        RADEC_JH.extend([[result[0]['RAJ2000'][i],result[0]['DEJ2000'][i],result[0]['Jmag'][i],result[0]['Hmag'][i]]])
    
RADEC_JH=np.array(RADEC_JH)    

# Delete stars duplicate (works only with numpy version >= 1.13.0)
RADEC_JH_unique=np.unique(RADEC_JH,axis=0)

SVOM_fov_Jmags=RADEC_JH_unique[:,2]
SVOM_fov_Hmags=RADEC_JH_unique[:,3]

# Conversion Vega to AB mag
#SVOM_fov_Jmags+=0.91
#SVOM_fov_Hmags+=1.39

# Keep only finite values
mask_J = np.isfinite(SVOM_fov_Jmags)
mask_H = np.isfinite(SVOM_fov_Hmags)

print ('Total number of stars in J band: %d' % len(SVOM_fov_Jmags[mask_J]))
print ('Total number of stars in H band: %d' % len(SVOM_fov_Hmags[mask_H]))


# completeness above 30 degrees galactic latitude. Vega magnitudes
completeness_catalog=[16,15,14.7]
completeness_dataset=[16.1,5.5,15.1]

In [ ]:
# Histograms number of saturated stars per FoV
plt.figure()
n, bins, patches = plt.hist(SVOM_fov_Jmags[mask_J], 100, facecolor='green', alpha=0.75)
plt.axvline(completeness_catalog[0],color='red',ls='--')
plt.xlabel(r'Magnitude (Vega)',size=14)
plt.ylabel('Counts',size=14)
plt.title('J band / total nb stars: %d' % (len(SVOM_fov_Jmags[mask_J])),size=14)
#plt.axis([40, 160, 0, 0.03])
#plt.xlim(0,4.5)
plt.tick_params(labelsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('plots/Distribution_J_mag.png')


In [ ]:

plt.figure()
n, bins, patches = plt.hist(SVOM_fov_Hmags[mask_H], 100, facecolor='green', alpha=0.75)
plt.axvline(completeness_catalog[1],color='red',ls='--')
plt.xlabel(r'Magnitude (Vega)',size=14)
plt.ylabel('Counts',size=14)
plt.title('Magnitude distribution in H band / total stars: %d' % (len(SVOM_fov_Hmags[mask_H])),size=14)
#plt.axis([40, 160, 0, 0.03])
#plt.xlim(0,4.5)
plt.tick_params(labelsize=12)
plt.grid(True)
plt.tight_layout()
plt.savefig('plots/Distribution_H_mag.png')

In [ ]:
np.min(SVOM_fov_Jmags), np.mean(SVOM_fov_Jmags), np.median(SVOM_fov_Jmags), np.max(SVOM_fov_Jmags)

In [ ]:
np.min(SVOM_fov_Hmags[mask_H]), np.mean(SVOM_fov_Hmags[mask_H]), np.median(SVOM_fov_Hmags[mask_H]), np.max(SVOM_fov_Hmags[mask_H])

In [ ]:
# Number of electrons to avoid persistence/saturation
Nelec=60000
Texps=[1.4,1.4*5,1.4*10,1.4*20,1.4*30,1.4*35]
#Texps=[1.4]

bands=['J','H']
seeing =0.8
moonage=7 # it does not matter

for Texp in Texps:
       for band in bands:
            list_saturated_stars=[]

            SB_saturaton, saturated_stars,nb_stars_sat,total_nb_stars,magsat=check_NIR_saturation(
                RADEC_SVOM_unique[:,0],RADEC_SVOM_unique[:,1],Texp,band,name_telescope='colibri',
                Nelec_saturation=Nelec,seeing=seeing,moon_age=moonage,verbose=True)

            nb_stars_sat=np.array(nb_stars_sat)
            total_nb_stars=np.array(total_nb_stars)
                        

            # Histograms number of saturated stars per FoV
            plt.figure()
            mask_nan = nb_stars_sat[:,2]>0
            n, bins, patches = plt.hist(np.log10(nb_stars_sat[:,2][mask_nan]), 50, facecolor='green', alpha=0.75)
            #n, bins, patches = plt.hist(nb_stars_sat[:,2], 50, facecolor='green', alpha=0.75)
            plt.xlabel(r'log$_{10}$ (Number of saturated stars per FoV)',size=14)
            plt.ylabel('Counts',size=14)
            plt.title('%s Band / %.2fs exposure / mag for sat.: %.2f' % (band,Texp,magsat),size=14)
            #plt.axis([40, 160, 0, 0.03])
            plt.xlim(0,4.5)
            plt.grid(True)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Histogram_nb_sat_stars_FoV_%s_%.2fs.png' % (band,Texp))

            # Cumulative distribution
            plt.figure()
            # adds 0 bins if any Fov with 0 saturating stars
            if len(nb_stars_sat[nb_stars_sat[:,2]==0]) > 0:
                bins2=np.insert(bins,1,0.00001)
                n2=np.insert(n,0,len(nb_stars_sat[nb_stars_sat[:,2]==0]))
            else:
                bins2=bins
                n2=n
            plt.plot((bins2[:-1]),100*np.cumsum(n2)/len(nb_stars_sat[:,2]))
            plt.xlabel(r'log$_{10}$ (Number of saturating stars per FoV)',size=14)
            plt.ylabel('Cumulative Fraction of FoV (in %)',size=14)
            plt.title('%s Band / %.2fs exposure / mag for sat.: %.2f \nFoV with 0 saturating stars: %d/%d' % (band,Texp,magsat,len(nb_stars_sat[nb_stars_sat[:,2]==0]),len(nb_stars_sat)),size=14)
            #plt.axis([40, 160, 0, 0.03])
            plt.ylim(0,100)
            plt.grid(True)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Cum_nb_sat_stars_FoV_%s_%.2fs.png' % (band,Texp))
            
            # Histograms number of saturated stars per FoV
            plt.figure()
            n, bins, patches = plt.hist(100*nb_stars_sat[:,2]/total_nb_stars,50 , facecolor='green', alpha=0.75)
            plt.xlabel(r'Fraction of saturated stars per FoV (in %)',size=14)
            plt.ylabel('Counts',size=14)
            plt.title('%s Band / %.2fs exposure / mag for sat.: %.2f' % (band,Texp,magsat),size=14)
            #plt.axis([40, 160, 0, 0.03])
            plt.xlim(0,100)
            plt.grid(True)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Histogram_nb_sat_stars_FoV_fraction_%s_%.2fs_120000.png' % (band,Texp))                       
            
            # Do not count the same star twice or more
            Saturated_stars_RADEC_unique=np.array([list(t) for t in set(tuple(element) for element in saturated_stars['RAJ2000','DEJ2000'])])
            
            plt.figure()
            n, bins, patches = plt.hist(Saturated_stars_RADEC_unique[:,0], 100, facecolor='green', alpha=0.75)
            plt.ylabel('Number of saturated stars',size=14)
            plt.xlabel('RA J2000 (degrees)',size=14)
            plt.title('Band %s / %.2fs exposure / %d bins' % (band,Texp,100),size=14)
            #plt.axis([40, 160, 0, 0.03])
            plt.grid(True)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Histogram_nb_sat_stars_RA_%s_%.2fs_120000.png' % (band,Texp))
                                    
            plt.figure()
            n, bins, patches = plt.hist(Saturated_stars_RADEC_unique[:,1], 100, facecolor='green', alpha=0.75)
            plt.ylabel('Number of saturated stars',size=14)
            plt.xlabel('DEC J2000 (degrees)',size=14)
            plt.title('Band %s / %.2fs exposure / %d bins' % (band,Texp,100),size=14)
            #plt.axis([40, 160, 0, 0.03])
            plt.grid(True)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Histogram_nb_sat_stars_DEC_%s_%.2fs_120000.png' % (band,Texp))
                                    

            color_map = plt.cm.Spectral_r 
            xbnds = np.array([ 180,-180.0]) 
            ybnds = np.array([-90.0,90.0]) 
            extent = [xbnds[0],xbnds[1],ybnds[0],ybnds[1]] 

            RA_result=coord.Angle(Saturated_stars_RADEC_unique[:,0]*u.degree)
            DEC_result=coord.Angle(Saturated_stars_RADEC_unique[:,1]*u.degree)

            RA_result2=RA_result.wrap_at(180*u.degree)

            fig = plt.figure(figsize=(8, 6)) 
            ax = fig.add_subplot(111) 
            x, y =  RA_result2,DEC_result
            gsize = (360,180) 
            image = plt.hexbin(x,y,cmap=color_map,gridsize=gsize,extent=extent,mincnt=1,bins='log',lw=3) 
            #image = plt.hist2d(x,y,cmap=color_map,bins=(360,180),cmin=1,norm=LogNorm())
            plt.scatter(Galactic_plane_RA2,Galactic_plane_DEC,color='red',label='galactic plane',s=0.3)

            ax.set_xlim(xbnds) 
            ax.set_ylim(ybnds)
            plt.title('Band %s / %.2fs exposure' % (band,Texp),size=14)
            plt.xlabel('RA J2000 (degrees)',size=14)  
            plt.ylabel('DEC J2000 (degrees)',size=14) 
            plt.grid(True) 
            cbar = plt.colorbar(image, spacing='uniform', extend='neither') 
            cbar.ax.set_ylabel(r'log$_{10}$ (Nb saturating stars per deg$^2$)',size=13)
            cbar.ax.tick_params(labelsize=12)
            plt.tick_params(labelsize=12)
            plt.tight_layout()
            plt.savefig('plots/Histogram_nb_sat_stars_RADEC_%s_%.2fs_120000.png' % (band,Texp))
            

In [ ]:
nb_stars_sat=np.array(nb_stars_sat)

In [ ]:
# Get the RA,DEC of the Field with the most and less saturating stars 
print ('max:',nb_stars_sat[np.argmax(nb_stars_sat[:,2])])
print ('min:',nb_stars_sat[np.argmin(nb_stars_sat[:,2])])

# III. Image simulations

Internet connection required here!   
The sources will be downloaded from the NOMAD-1 catalogue using astroquery

## 1) Parameters

In [ ]:
seeing=1
moonage=7
bands=['J','H']
Texps=[1.4,1.4*5,1.4*10,1.4*20,1.4*30,1.4*35]

# RADEC: 1st in crowded field, 2nd in sparse field
# crowded: 26121 saturated in H band for 49s
# sparse: 110 saturated stars in H band for 49s
RA_image=[266.4083,159.3031719]   
DEC_image=[-29.0064,33.71224656]    
#RA_image=[260,30]
#DEC_image=[-30,30]

output_dir = 'CAGIRE_saturation'
configFile = 'CAGIRE_saturation.hjson'

## 2) Make PSFs for each selected band

In [ ]:
from pyETC.pyETC import etc
from ImSimpy.utils.PSFUtils import createPSF, convolvePSF

colibri_ETC=etc(configFile=configFile,name_telescope='colibri')

#Parameters
imsize=[256,256]
oversampling=1
oversamp2=15
beta=2

# Chek which bands are considered
bands = ['J','H']

name_inst=path+'/data/psf/instrument/%s/instrument' % output_dir
name_atm=path+'/data/psf/atmosphere/%s/atmosphere' % output_dir


for band in bands:
    colibri_ETC.information['channel']='CAGIRE'
    
    colibri_ETC.information['filter_band']=band
    
    # Seeing computed for an airmass of 1.5 and 0.8" at 500nm
    if band == 'J': seeing = 0.85
    elif band == 'H': seeing = 0.8
    
    # Compute colibri_ETC to get the seeing along the line of sight scale to airmass and wavelength
    colibri_ETC.sim()
    
    pixsize=colibri_ETC.information['cameras'][colibri_ETC.information['channel']]['Photocell_SizeX']
    pixscale=colibri_ETC.information['pixelScale_X']    # assume same in Y
    wvl=colibri_ETC.information['effWavelength']
    DM1=colibri_ETC.information['D_M1']
    DM2=colibri_ETC.information['D_M2']
    f_length=colibri_ETC.information['foc_len']

    print ('band: %s   PixSize: %.2f   PixScale: %.2f   wvl_eff: %.2f   \nDM1: %.2f   DM2: %.2f    f_Length: %.2f  seeing: %.2f' % (band,pixsize*1e6,pixscale,wvl,DM1,DM2,f_length,seeing))
    
    # Compute Atmosphere PSF using moffat
    PSF_type='moffat'
    name_atm_band=name_atm+'_%s.fits' % band
    createPSF(filename=name_atm_band,PSF_type=PSF_type,imsize=imsize,pixel_size=[pixsize,pixsize],
                   pixel_scale=pixscale,eff_wvl=wvl,seeing=seeing,DM1=DM1,DM2=DM2,focal_length=f_length,
                   oversamp=oversampling,oversamp2=oversamp2,beta=beta,disp=False,unsigned16bit=False)
    
    # Compute INstrumental PSF using ideal AIry function
    PSF_type='airy'
    name_inst_band=name_inst+'_%s.fits' % band
    createPSF(filename=name_inst_band,PSF_type=PSF_type,imsize=imsize,pixel_size=[pixsize,pixsize],
                   pixel_scale=pixscale,eff_wvl=wvl,seeing=seeing,DM1=DM1,DM2=DM2,focal_length=f_length,
                   oversamp=oversampling,oversamp2=oversamp2,beta=beta,disp=False,unsigned16bit=False)
    
    # Convolve Total PSF
    name_PSF_total=path+'/data/psf/total_PSF/%s/PSF_total_%s.fits' % (output_dir,band)
    convolvePSF(filename1=name_atm_band,filename2=name_inst_band,filename3=name_PSF_total)
    

## 3) Create gain map 

In [ ]:
from ImSimpy.utils.generateCalib import Offset, Vignetting, GainMap

GainMap(filename=path+'/data/GainMap/%s/Gain_nir.fits' % output_dir,
        Type='random',mean=1.3, std=0.05,Nampl=32,xsize=2048,ysize=2048)

## 4) Create images

Create simple images without artefacts

In [ ]:
from ImSimpy.ImSimpy import ImageSimulator

colibri_IS=ImageSimulator(configFile=configFile,name_telescope='colibri')

#Read the configfile
colibri_IS.readConfigs()

colibri_IS.config['seeing_zenith']=seeing
colibri_IS.config['moon_age']=moonage

# Just 1 exposure means that the RN=24e- are added just once (may be it is wrong?)
colibri_IS.config['Nexp']=1


colibri_IS.config['SourcesList']['generate']['radius']=21.7
colibri_IS.config['SourcesList']['generate']['catalog']='II/246'

# Load existing PSF instead of computing new ones for speeding up the calculations
colibri_IS.config['PSF']['total']['method']='load'

for ra,dec in zip(RA_image,DEC_image):
    colibri_IS.config['SourcesList']['generate']['RA']=ra
    colibri_IS.config['SourcesList']['generate']['DEC']=dec
    colibri_IS.config['RA']=ra
    colibri_IS.config['DEC']=dec

    for band in bands:
        colibri_IS.config['filter_band']=band
            
        colibri_IS.config['PSF']['total']['file']='total_PSF/%s/PSF_total_%s.fits' % (output_dir,band)
        
        for Texp in Texps:
            colibri_IS.config['exptime']=Texp
            # filename of the output image
            colibri_IS.config['output']='CAGIRE_saturation/%.0f_%.0f_%s_%.0f.fits' % (ra,dec,band,Texp)
            # Run the Image Simulator
            colibri_IS.simulate('data')


# Visualisation

In [ ]:
bands=['J','H']
Texps=[1.4,1.4*20,1.4*35]
Nelec_saturation=60000
# RADEC: 1st in crowded field, 2nd in sparse field
seeing=0.8
moonage=7

RA_image=[266.4083,159.3031719]   
DEC_image=[-29.0064,33.71224656]
#RA_image=[260,30]
#DEC_image=[-30,30]


In [ ]:
from astropy.wcs import WCS
from matplotlib.colors import LogNorm
from astropy.visualization import MinMaxInterval, SqrtStretch,LogStretch,AsinhStretch, LinearStretch,ImageNormalize,ZScaleInterval

for ra,dec in zip(RA_image,DEC_image):
    for band in bands:
        for Texp in Texps:
            
            # Get the number of saturated stars and the total number of stars on the field
            SB_saturaton, saturated_stars,nb_stars_sat,total_nb_stars,magsat=check_NIR_saturation(
                ra,dec,Texp,band,name_telescope='colibri',Nelec_saturation=Nelec_saturation,
                seeing=seeing,moon_age=moonage,verbose=True)
            
            # filename of the output image
            fits_filename=path+'/images/CAGIRE_saturation/%.0f_%.0f_%s_%.0f.fits' % (ra,dec,band,Texp)
            
            # Extract header and data from the fits file
            image,header = fits.getdata(fits_filename,header=True)

            # Compute mean and standard deviation of the image
            #image_mean = np.mean(image)
            #image_std = np.std(image)

            # Compute limits for scaling the colorbar (arbitrary, use your own prefered values)
            #vmin = image_mean - 1*image_std
            #vmax = image_mean + 1*image_std
            
            #norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=SqrtStretch())
            
            # Based on IRAF's zscale
            vmin,vmax=ZScaleInterval(nsamples=1000, contrast=0.25, max_reject=0.5,
                                     min_npixels=5, krej=2.5, max_iterations=5).get_limits(image)
            #norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())
            #norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=SqrtStretch())
            #norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=AsinhStretch())
            norm = ImageNormalize(vmin=vmin, vmax=vmax, stretch=LinearStretch())

            
            # Use astropy.wcs.WCS to extract coordinates
            wcs = WCS(header)

            # Visualize image with WCS coordinates
            plt.figure(figsize=(7,7))
            plt.subplot(projection=wcs)
            plt.imshow(image,interpolation='none',cmap='gray',vmin=vmin,vmax=vmax,origin='lower')
            plt.imshow(image,interpolation='none',cmap='gray',origin='lower',norm=norm)
            plt.grid(color='white', ls='--')
            plt.title('%s band / %.2fs exposure \n saturation reached for %d / %d stars'  % (band,Texp,nb_stars_sat[0][2],total_nb_stars[0]),size=15)
            plt.xlabel('Right Ascension (J2000)',size=14)
            plt.ylabel('Declination (J2000)',size=14)
            plt.tick_params(labelsize=14)
            #plt.tight_layout()
            plt.savefig('plots/images_RA%.2f_DE%.2f_%s_%.2fs.png' % (ra,dec,band,Texp))